In [1]:
import os
import zipfile as zf
import pandas as pd
import numpy as np
import cvm
pd.set_option("max_colwidth", 10)

In [2]:
df = cvm.load_metadata()
df

,DT_REFER,VERSAO,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB
0,2010-1...,1,1023,DFP,4822,2011-0...
1,2010-1...,2,1023,DFP,4823,2011-0...
2,2010-1...,3,1023,DFP,6945,2011-0...
3,2010-1...,1,14206,DFP,5436,2011-0...
4,2010-1...,2,14206,DFP,6775,2011-0...
...,...,...,...,...,...,...
34253,2021-0...,1,4693,ITR,110135,2021-1...
34254,2021-0...,1,21091,ITR,103186,2021-0...
34255,2021-0...,1,21091,ITR,106768,2021-0...
34256,2021-0...,1,21091,ITR,109262,2021-1...


In [3]:
files_names = sorted(os.listdir('data/'))
files_names

['dfp_cia_aberta_2010.zip',
 'dfp_cia_aberta_2011.zip',
 'dfp_cia_aberta_2012.zip',
 'dfp_cia_aberta_2013.zip',
 'dfp_cia_aberta_2014.zip',
 'dfp_cia_aberta_2015.zip',
 'dfp_cia_aberta_2016.zip',
 'dfp_cia_aberta_2017.zip',
 'dfp_cia_aberta_2018.zip',
 'dfp_cia_aberta_2019.zip',
 'dfp_cia_aberta_2020.zip',
 'dfp_cia_aberta_2021.zip',
 'itr_cia_aberta_2011.zip',
 'itr_cia_aberta_2012.zip',
 'itr_cia_aberta_2013.zip',
 'itr_cia_aberta_2014.zip',
 'itr_cia_aberta_2015.zip',
 'itr_cia_aberta_2016.zip',
 'itr_cia_aberta_2017.zip',
 'itr_cia_aberta_2018.zip',
 'itr_cia_aberta_2019.zip',
 'itr_cia_aberta_2020.zip',
 'itr_cia_aberta_2021.zip',
 'itr_cia_aberta_2022.zip']

In [4]:
def df_format(df) -> pd.DataFrame:
    # df.MOEDA.value_counts()
    # REAL    43391302
    df.drop(columns=['MOEDA'], inplace=True)
    
    # df.ESCALA_MOEDA.value_counts()
    # MIL        40483230
    # UNIDADE     2908072
    df.ESCALA_MOEDA = df.ESCALA_MOEDA.map({'MIL': 1000, 'UNIDADE': 1})

    # unit base currency
    df.VL_CONTA = df.VL_CONTA * df.ESCALA_MOEDA
    df.drop(columns=['ESCALA_MOEDA'], inplace=True)

    # df.ST_CONTA_FIXA.unique() -> ['S', 'N']
    df.ST_CONTA_FIXA = df.ST_CONTA_FIXA.map({'S': True, 'N': False})

    # df.ORDEM_EXERC.unique() -> ['PENÚLTIMO', 'ÚLTIMO']
    df.ORDEM_EXERC = df.ORDEM_EXERC.map({'ÚLTIMO': 0, 'PENÚLTIMO': -1})
    df.ORDEM_EXERC = df.ORDEM_EXERC.astype(np.int8)

    # BPA, BPP and DFC files have no DT_INI_EXERC column
    if 'DT_INI_EXERC' in df.columns:
        df.DT_INI_EXERC = pd.to_datetime(df.DT_INI_EXERC)

    return df 

In [5]:
columns_type = {
    'CNPJ_CIA': str,
    'VERSAO': np.uint8, # unique -> ['3', '2', '4', '1', '7', '5', '6', '9', '8']
    'CD_CVM': np.uint32, # max < 600_000
    'VL_CONTA': float,
}
# date_columns = ['DT_REFER', 'DT_FIM_EXERC', 'DT_INI_EXERC']
date_columns = ['DT_REFER', 'DT_FIM_EXERC']
kwargs = {
    'sep': ';',
    'encoding': 'iso-8859-1',
    'dtype': columns_type,
    'parse_dates': date_columns}

In [6]:
# full scan = 8m 10s -> 5.6 GB df memory space
# formated colums = 4m 30s -> 3.5 GB df memory space
df = pd.DataFrame()
for file_name in files_names[:]:
    print(file_name)
    file_path = 'data/' + file_name
    parent_file = zf.ZipFile(file_path)
    # print(parent_file.namelist())
    children_files_names = parent_file.namelist()
    for child_file_name in children_files_names[1:]:
        # print(child_file_name)
        child_file = parent_file.open(child_file_name)
        df_child = pd.read_csv(child_file, **kwargs)
        df_child = df_format(df_child)
        df = pd.concat([df, df_child], ignore_index=True)

dfp_cia_aberta_2010.zip
dfp_cia_aberta_2011.zip
dfp_cia_aberta_2012.zip
dfp_cia_aberta_2013.zip
dfp_cia_aberta_2014.zip
dfp_cia_aberta_2015.zip
dfp_cia_aberta_2016.zip
dfp_cia_aberta_2017.zip
dfp_cia_aberta_2018.zip
dfp_cia_aberta_2019.zip
dfp_cia_aberta_2020.zip
dfp_cia_aberta_2021.zip
itr_cia_aberta_2011.zip
itr_cia_aberta_2012.zip
itr_cia_aberta_2013.zip
itr_cia_aberta_2014.zip
itr_cia_aberta_2015.zip
itr_cia_aberta_2016.zip
itr_cia_aberta_2017.zip
itr_cia_aberta_2018.zip
itr_cia_aberta_2019.zip
itr_cia_aberta_2020.zip
itr_cia_aberta_2021.zip
itr_cia_aberta_2022.zip


In [9]:
df

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF
0,00.000...,2010-12-31,3,BCO BR...,1023,DF Con...,-1,2009-12-31,1,Ativo ...,7.0257...,True,NaT,NaN
1,00.000...,2010-12-31,3,BCO BR...,1023,DF Con...,0,2010-12-31,1,Ativo ...,8.0281...,True,NaT,NaN
2,00.000...,2010-12-31,3,BCO BR...,1023,DF Con...,-1,2009-12-31,1.01,Caixa ...,3.1037...,True,NaT,NaN
3,00.000...,2010-12-31,3,BCO BR...,1023,DF Con...,0,2010-12-31,1.01,Caixa ...,2.5219...,True,NaT,NaN
4,00.000...,2010-12-31,3,BCO BR...,1023,DF Con...,-1,2009-12-31,1.01.01,Caixa ...,8.0986...,False,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43391297,31.701...,2022-03-31,1,ATHENA...,25810,DF Ind...,0,2022-03-31,7.08.0...,Divide...,0.0000...,True,2022-01-01,NaN
43391298,31.701...,2022-03-31,1,ATHENA...,25810,DF Ind...,-1,2021-03-31,7.08.0...,Lucros...,0.0000...,True,2021-01-01,NaN
43391299,31.701...,2022-03-31,1,ATHENA...,25810,DF Ind...,0,2022-03-31,7.08.0...,Lucros...,-2.5867...,True,2022-01-01,NaN
43391300,31.701...,2022-03-31,1,ATHENA...,25810,DF Ind...,-1,2021-03-31,7.08.05,Outros,0.0000...,True,2021-01-01,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43391302 entries, 0 to 43391301
Data columns (total 14 columns):
 #   Column         Dtype         
---  ------         -----         
 0   CNPJ_CIA       object        
 1   DT_REFER       datetime64[ns]
 2   VERSAO         uint8         
 3   DENOM_CIA      object        
 4   CD_CVM         uint32        
 5   GRUPO_DFP      object        
 6   ORDEM_EXERC    int8          
 7   DT_FIM_EXERC   datetime64[ns]
 8   CD_CONTA       object        
 9   DS_CONTA       object        
 10  VL_CONTA       float64       
 11  ST_CONTA_FIXA  bool          
 12  DT_INI_EXERC   datetime64[ns]
 13  COLUNA_DF      object        
dtypes: bool(1), datetime64[ns](3), float64(1), int8(1), object(6), uint32(1), uint8(1)
memory usage: 3.5+ GB


In [31]:
df.GRUPO_DFP.unique()
# df

array(['DF Consolidado - Balanço Patrimonial Ativo',
       'DF Individual - Balanço Patrimonial Ativo',
       'DF Consolidado - Balanço Patrimonial Passivo',
       'DF Individual - Balanço Patrimonial Passivo',
       'DF Consolidado - Demonstração do Fluxo de Caixa (Método Direto)',
       'DF Individual - Demonstração do Fluxo de Caixa (Método Direto)',
       'DF Consolidado - Demonstração do Fluxo de Caixa (Método Indireto)',
       'DF Individual - Demonstração do Fluxo de Caixa (Método Indireto)',
       'DF Consolidado - Demonstração das Mutações do Patrimônio Líquido',
       'DF Individual - Demonstração das Mutações do Patrimônio Líquido',
       'DF Consolidado - Demonstração de Resultado Abrangente',
       'DF Individual - Demonstração de Resultado Abrangente',
       'DF Consolidado - Demonstração do Resultado',
       'DF Individual - Demonstração do Resultado',
       'DF Consolidado - Demonstração de Valor Adicionado',
       'DF Individual - Demonstração de Valor A